In [22]:
# Install dependencies
!pip install -r requirements.txt

In [23]:
"""
Download geckodriver here: https://github.com/mozilla/geckodriver/releases
And add exe file in this folder
"""
import os
import sys
os.path.dirname(sys.executable)

'/home/lilian/scrapping-python/env/bin'

In [24]:
# Import dependencies
from selenium import webdriver
from selenium.webdriver.firefox.firefox_binary import FirefoxBinary
from selenium.webdriver.common.by import By

In [25]:
# Change with your firefox bin
firefox_bin = '/usr/bin/firefox'

In [29]:
# Init driver
binary = FirefoxBinary(firefox_bin)
driver = webdriver.Firefox(firefox_binary=binary)

# Get item page
driver.get("https://bindingofisaacrebirth.fandom.com/wiki/Items")

# Wait for load all items
driver.implicitly_wait(150)

# Get each items in an array
rows = driver.find_elements(By.CLASS_NAME, 'row-collectible')
print(rows[700].text)

# Close driver
driver.close()

/home/lilian/scrapping-python/env/lib/python3.7/site-packages/ipykernel_launcher.py:3: DeprecationWarning: firefox_binary has been deprecated, please pass in a Service object
  This is separate from the ipykernel package so we can avoid doing imports until


Strawman 5.100.667 A helping hand Spawns a controllable Keeper to fight enemies alongside Isaac, functioning similarly to Jacob and Esau. If Keeper dies, he spawns 5 blue spiders and Strawman is removed from Isaac's inventory. 2
